### Monta Tabela de Anotação

In [11]:
import collections
import re
import pandas as pd

dictF = collections.OrderedDict()
anotador = 0
cont = 1
str1 = ""

for filename in ['Clear_01_1_ISO_Andressa_Bressani.xml','Clear_01_1_ISO_Eric_Eltz.xml', 'Clear_01_1_ISO_Keli_Schell.xml' ]:
    
    with open(filename,'r') as file:
        cont = 1
        for line in file:

            if line.find("EM-ID") >= 0 and line.find("/EM") >= 0:
                str1 = re.findall('(?<=>).*?(?=<)', line)[0] + " - " + str(cont)
                categoria = re.findall('(?<=CATEG=").*?(?=")', line)[0]
                #print(str1)
                if str1 in dictF:
                    dictF[str1][anotador] = categoria
                else:
                    dictF[str1] = ['','','']
                    dictF[str1][anotador] = categoria
            
            elif line.find("EM-ID") >= 0:
                str1 = re.findall('(?<=>).*?(?=$)', line)[0]
                str1 = str1.replace("\n", "")
                categoria = re.findall('(?<=CATEG=").*?(?=")', line)[0]
            
            elif line.find("/EM") >= 0:
                str1 = str1 + " " + re.findall('(?<=^).*?(?=<)', line)[0] + " - " + str(cont)
                str1 = str1.replace("\n", "")
                if str1 in dictF:
                    dictF[str1][anotador] = categoria
                else:
                    dictF[str1] = ['','','']
                    dictF[str1][anotador] = categoria
                str1 = ""
            
            else:
                str1 = str1 + " " + line
                
            cont += 1
        
    anotador += 1
    
print(len(dictF))
    
kappa_pd = pd.DataFrame.from_dict(dictF,orient='index')
kappa_pd.sample(10)

286


,0,1,2
falhas transcorrentes - 902,CONTEXTOgeologicoDeBACIA,,CONTEXTOgeologicoDeBACIA
Setuva - 813,,CONTEXTOgeologicoDeBACIA,
diques - 2347,,,outro
Tranqueira - 817,,CONTEXTOgeologicoDeBACIA,
pré-cambrianas - 4095,eon,eon,eon
convergência intercontinental - 966,,,CONTEXTOgeologicoDeBACIA
534 ± 16 Ma - 1080,,,idade
planos de falha - 1444,,,CONTEXTOgeologicoDeBACIA
Cisalhamento - 3475,,CONTEXTOgeologicoDeBACIA,
cisalhamento - 4489,,CONTEXTOgeologicoDeBACIA,


### Montagem da Tabela para o Kappa

In [2]:
classes = set()

for x in kappa_pd[0].unique():
    classes.add(x.lower())
    
for x in kappa_pd[1].unique():
    classes.add(x.lower())
    
for x in kappa_pd[2].unique():
    classes.add(x.lower())

classes

{'',
 'baciasedimentar',
 'contextogeologicodebacia',
 'eon',
 'era',
 'formacao',
 'idade',
 'outro',
 'periodo',
 'unidadeestratigrafica'}

In [16]:
w, h = len(classes), len(kappa_pd);
kappa_table = [[0 for x in range(w)] for y in range(h)] 

row_id = 0
for index, row in kappa_pd.iterrows():
    class_id = 0
    for clas in classes:
        if row[0].lower() == clas:
            kappa_table[row_id][class_id] += 1
        if row[1].lower() == clas:
            kappa_table[row_id][class_id] += 1
        if row[2].lower() == clas:
            kappa_table[row_id][class_id] += 1
        class_id += 1
    row_id += 1
            
kappa_table[:10] #sample 10

[[1, 2, 0, 0, 0, 0, 0, 0, 0, 0],
 [0, 0, 2, 0, 0, 0, 0, 0, 0, 1],
 [0, 0, 0, 0, 0, 0, 0, 0, 3, 0],
 [0, 0, 0, 0, 0, 0, 0, 0, 0, 3],
 [1, 2, 0, 0, 0, 0, 0, 0, 0, 0],
 [1, 2, 0, 0, 0, 0, 0, 0, 0, 0],
 [0, 0, 0, 0, 0, 0, 0, 0, 3, 0],
 [1, 2, 0, 0, 0, 0, 0, 0, 0, 0],
 [1, 2, 0, 0, 0, 0, 0, 0, 0, 0],
 [0, 0, 0, 0, 0, 0, 0, 0, 0, 3]]

In [4]:
from statsmodels.stats.inter_rater import fleiss_kappa
fleiss_kappa(kappa_table)

0.15955482017731876

### Cálculo do Fleiss Kappa manualmente

In [18]:
NC = 3 * len(kappa_table)

Z = 0 
for vec in kappa_table:
    S = 0
    for clas in vec:
        S += (clas * (clas - 1))
    S = (1 / (3 * (3 - 1))) * S
    if Z < 10: # sample
        print(vec)
        print(S)
    Z += S

[1, 2, 0, 0, 0, 0, 0, 0, 0, 0]
0.3333333333333333
[0, 0, 2, 0, 0, 0, 0, 0, 0, 1]
0.3333333333333333
[0, 0, 0, 0, 0, 0, 0, 0, 3, 0]
1.0
[0, 0, 0, 0, 0, 0, 0, 0, 0, 3]
1.0
[1, 2, 0, 0, 0, 0, 0, 0, 0, 0]
0.3333333333333333
[1, 2, 0, 0, 0, 0, 0, 0, 0, 0]
0.3333333333333333
[0, 0, 0, 0, 0, 0, 0, 0, 3, 0]
1.0
[1, 2, 0, 0, 0, 0, 0, 0, 0, 0]
0.3333333333333333
[1, 2, 0, 0, 0, 0, 0, 0, 0, 0]
0.3333333333333333
[0, 0, 0, 0, 0, 0, 0, 0, 0, 3]
1.0
[0, 0, 0, 0, 0, 0, 0, 0, 3, 0]
1.0
[0, 0, 0, 0, 0, 0, 0, 0, 0, 3]
1.0
[0, 0, 0, 0, 0, 0, 0, 0, 3, 0]
1.0
[0, 0, 0, 0, 0, 0, 0, 0, 0, 3]
1.0


In [6]:
PA = Z / len(kappa_table)
PA

0.43006993006992733

In [7]:
NC = 3 * len(kappa_table)
NC

858

In [8]:
class_sum = pd.DataFrame(data=kappa_table).sum(axis=0)
class_sum

0    417
1    233
2     26
3      1
4      2
5      5
6     30
7     42
8     61
9     41
dtype: int64

In [9]:
PE = 0
for clas in class_sum:
    print(str(clas) + "/" +str(NC))
    PE += (clas/NC) * (clas/NC)
PE

417/858
233/858
26/858
1/858
2/858
5/858
30/858
42/858
61/858
41/858


0.32187121347960507

In [10]:
K = (PA - PE) / (1 - PE)
K

0.15955482017731473

### Calculo Kappa de Identificação

In [23]:
w, h = 2, len(kappa_pd);
kappa_id_table = [[0 for x in range(w)] for y in range(h)]

row_id = 0
for index, row in kappa_pd.iterrows():
    class_id = 0
    for clas in classes:
        if row[0].lower() == clas:
            kappa_id_table[row_id][class_id] += 1
        if row[1].lower() == clas:
            kappa_id_table[row_id][class_id] += 1
        if row[2].lower() == clas:
            kappa_id_table[row_id][class_id] += 1
        if class_id == 0:
            class_id += 1
    row_id += 1
    
kappa_id_table

[[1, 2],
 [0, 3],
 [0, 3],
 [0, 3],
 [1, 2],
 [1, 2],
 [0, 3],
 [1, 2],
 [1, 2],
 [0, 3],
 [0, 3],
 [0, 3],
 [0, 3],
 [0, 3],
 [1, 2],
 [1, 2],
 [1, 2],
 [1, 2],
 [1, 2],
 [1, 2],
 [1, 2],
 [0, 3],
 [0, 3],
 [0, 3],
 [1, 2],
 [0, 3],
 [0, 3],
 [0, 3],
 [1, 2],
 [1, 2],
 [2, 1],
 [2, 1],
 [1, 2],
 [0, 3],
 [1, 2],
 [1, 2],
 [2, 1],
 [0, 3],
 [0, 3],
 [0, 3],
 [0, 3],
 [0, 3],
 [0, 3],
 [0, 3],
 [0, 3],
 [0, 3],
 [1, 2],
 [0, 3],
 [0, 3],
 [1, 2],
 [0, 3],
 [0, 3],
 [0, 3],
 [0, 3],
 [1, 2],
 [1, 2],
 [1, 2],
 [1, 2],
 [0, 3],
 [1, 2],
 [1, 2],
 [0, 3],
 [0, 3],
 [0, 3],
 [1, 2],
 [0, 3],
 [0, 3],
 [1, 2],
 [1, 2],
 [0, 3],
 [0, 3],
 [1, 2],
 [0, 3],
 [1, 2],
 [0, 3],
 [1, 2],
 [0, 3],
 [0, 3],
 [0, 3],
 [1, 2],
 [0, 3],
 [0, 3],
 [0, 3],
 [1, 2],
 [0, 3],
 [1, 2],
 [0, 3],
 [0, 3],
 [2, 1],
 [1, 2],
 [2, 1],
 [2, 1],
 [2, 1],
 [2, 1],
 [2, 1],
 [2, 1],
 [1, 2],
 [2, 1],
 [2, 1],
 [2, 1],
 [2, 1],
 [2, 1],
 [2, 1],
 [2, 1],
 [2, 1],
 [1, 2],
 [2, 1],
 [2, 1],
 [2, 1],
 [2, 1],
 [2, 1],
 

In [22]:
fleiss_kappa(kappa_id_table)

-0.10109463449648438

### Calculo Kappa da Classificação

In [39]:
kappa_class_table = pd.DataFrame(data=kappa_table)
kappa_class_table = kappa_class_table[kappa_class_table[0]==0]
kappa_class_table.drop(kappa_class_table.columns[0],inplace=True,axis=1)
kappa_class_table.shape

(50, 9)

In [40]:
fleiss_kappa(kappa_class_table.as_matrix())

0.8911202516331963